In [25]:
from pydub import AudioSegment
import numpy as np
import scipy.fftpack

def get_frequencies_and_amplitudes_from_audio(file_path, dt=1.0, rate=44100):
    """
    Returns a list of tuples containing frequencies and their amplitudes in the audio file at intervals of dt seconds.

    :param file_path: Path to the audio file
    :param dt: Time interval in seconds to analyze frequencies
    :param rate: Sample rate (samples per second, default is 44100)
    :return: List of tuples (frequencies, amplitudes)
    """
    # Load audio file
    audio = AudioSegment.from_file(file_path)
    
    # Convert audio to mono and to raw audio data
    audio = audio.set_frame_rate(rate).set_channels(1)
    samples = np.array(audio.get_array_of_samples())
    
    # Calculate the number of samples per dt interval
    num_samples_per_dt = int(rate * dt)
    
    results = []

    for start in range(0, len(samples), num_samples_per_dt):
        end = start + num_samples_per_dt
        segment = samples[start:end]

        if len(segment) < num_samples_per_dt:
            break  # Stop if there are not enough samples for a full segment

        # Apply FFT - Fast Fourier Transform
        fft_data = np.abs(scipy.fftpack.fft(segment))
        freqs = np.fft.fftfreq(len(fft_data), 1.0 / rate)
        
        # Filter out negative frequencies
        positive_freqs = freqs[:len(freqs)//2]
        positive_amplitudes = fft_data[:len(fft_data)//2]

        # Append frequencies and their corresponding amplitudes
        results.append((positive_freqs, positive_amplitudes))

    return results

# Example usage
file_path = ""
dt = 1.0  # 1 second intervals
results = get_frequencies_and_amplitudes_from_audio(file_path, dt)

Listening...
Peak Frequency: 775.1953125 Hz
Peak Frequency: 4780.37109375 Hz
Peak Frequency: 6459.9609375 Hz
Peak Frequency: 4177.44140625 Hz
Peak Frequency: 4823.4375 Hz
Peak Frequency: 3402.24609375 Hz
Peak Frequency: 2583.984375 Hz
Peak Frequency: 3316.11328125 Hz
Peak Frequency: 1851.85546875 Hz
Peak Frequency: 2024.12109375 Hz
Peak Frequency: 2885.44921875 Hz
Peak Frequency: 3919.04296875 Hz
Peak Frequency: 2024.12109375 Hz
Peak Frequency: 4134.375 Hz
Peak Frequency: 6072.36328125 Hz
Peak Frequency: 2368.65234375 Hz
Peak Frequency: 4005.17578125 Hz
Peak Frequency: 3014.6484375 Hz
Peak Frequency: 3186.9140625 Hz
Peak Frequency: 2885.44921875 Hz
Peak Frequency: 5254.1015625 Hz
Peak Frequency: 5124.90234375 Hz
Peak Frequency: 3789.84375 Hz
Peak Frequency: 4521.97265625 Hz
Peak Frequency: 4521.97265625 Hz
Peak Frequency: 1464.2578125 Hz
Peak Frequency: 5943.1640625 Hz
Peak Frequency: 3014.6484375 Hz
Peak Frequency: 6072.36328125 Hz
Peak Frequency: 2239.453125 Hz
Peak Frequency: 4220.5

In [ ]:
import pyaudio
import numpy as np
import time

def play_frequencies(frequencies, rate=44100):
    """
    Plays a list of frequencies, each for one second.

    :param frequencies: List of frequencies in Hz
    :param rate: Sample rate (samples per second, default is 44100)
    """
    p = pyaudio.PyAudio()
    
    # Function to generate a sine wave for a given frequency
    def generate_sine_wave(freq, duration, rate):
        t = np.linspace(0, duration, int(rate * duration), endpoint=False)
        wave = 0.5 * np.sin(2 * np.pi * freq * t)
        return wave.astype(np.float32)
    
    stream = p.open(format=pyaudio.paFloat32,
                    channels=1,
                    rate=rate,
                    output=True)
    
    for freq in frequencies:
        print(f"Playing frequency: {freq} Hz")
        wave = generate_sine_wave(freq, 1, rate)
        stream.write(wave.tobytes())
        time.sleep(1)  # Wait for 1 second

    stream.stop_stream()
    stream.close()
    p.terminate()

# Example usage
frequencies = [10000] * 100  # List of frequencies to play
play_frequencies(frequencies)
